In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [10]:
import joblib

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [12]:
data = pd.read_csv('ts_Final_data.csv')

In [13]:
data

,date,serial_number,model,capacity_bytes,failure,smart_5_raw,smart_9_raw,smart_12_raw,smart_187_raw,smart_188_raw,smart_190_raw,smart_193_raw,smart_194_raw,smart_197_raw,smart_198_raw,smart_199_raw,smart_241_raw,smart_242_raw,days_to_failure
0,2023-01-01,W300T02T,ST4000DM000,4000787030016,0,5392.0,66939.0,9.0,21.0,0.0,18.0,11740.0,18.0,0.0,0.0,0.0,6.348879e+10,3.674318e+11,45
1,2023-01-01,W300R7SP,ST4000DM000,4000787030016,0,8.0,66944.0,5.0,45.0,0.0,19.0,11776.0,19.0,0.0,0.0,0.0,6.180113e+10,3.769783e+11,67
2,2023-01-01,W300R8BM,ST4000DM000,4000787030016,0,0.0,66938.0,11.0,0.0,0.0,18.0,11867.0,18.0,0.0,0.0,0.0,6.297468e+10,3.565669e+11,139
3,2023-01-01,Z302CSM2,ST4000DM000,4000787030016,0,48.0,60913.0,11.0,34.0,20.0,17.0,10396.0,17.0,0.0,0.0,36.0,5.442403e+10,3.333510e+11,57
4,2023-01-01,WD-WX71A74PAYDF,WDC WD5000LPVX,500107862016,0,0.0,71225.0,13.0,0.0,0.0,0.0,3480.0,22.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647213,2023-12-31,X270A01MFVKG,TOSHIBA MG08ACA16TA,16000900661248,0,0.0,286.0,15.0,0.0,0.0,0.0,32.0,37.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,3
1647214,2023-12-31,X2E0A060FVKG,TOSHIBA MG08ACA16TA,16000900661248,0,0.0,284.0,15.0,0.0,0.0,0.0,34.0,36.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,87
1647215,2023-12-31,8230A182FVKG,TOSHIBA MG08ACA16TA,16000900661248,0,0.0,7484.0,28.0,0.0,0.0,0.0,53.0,41.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,28
1647216,2023-12-31,8230A13GFVKG,TOSHIBA MG08ACA16TA,16000900661248,0,0.0,7474.0,20.0,0.0,0.0,0.0,42.0,32.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,75


In [14]:
data.sort_values(by=['serial_number', 'date'], inplace=True)

In [15]:
le = LabelEncoder()
data['model'] = le.fit_transform(data['model'])

In [16]:
features = ['model', 'capacity_bytes', 'failure', 'smart_5_raw', 'smart_9_raw', 'smart_12_raw',
            'smart_187_raw', 'smart_188_raw', 'smart_190_raw', 'smart_193_raw',
            'smart_194_raw', 'smart_197_raw', 'smart_198_raw', 'smart_199_raw',
            'smart_241_raw', 'smart_242_raw']
target = 'days_to_failure'

In [17]:
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

In [43]:
def create_sequences(data, target_col, window_size=30):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:i+window_size][features].values)
        y.append(data.iloc[i+window_size][target_col])
    return np.array(X), np.array(y)

In [44]:
X, y = create_sequences(data, target, window_size=30)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

In [5]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))

c:\Users\asas7\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [7]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/5
37062/37062 ━━━━━━━━━━━━━━━━━━━━ 610s 16ms/step - loss: 18963.0117 - val_loss: 11914.4756
Epoch 2/5
37062/37062 ━━━━━━━━━━━━━━━━━━━━ 607s 16ms/step - loss: 11732.4473 - val_loss: 10991.8213
Epoch 3/5
37062/37062 ━━━━━━━━━━━━━━━━━━━━ 594s 16ms/step - loss: 10804.9336 - val_loss: 10251.7041
Epoch 4/5
37062/37062 ━━━━━━━━━━━━━━━━━━━━ 640s 17ms/step - loss: 10093.3809 - val_loss: 9663.8916
Epoch 5/5
37062/37062 ━━━━━━━━━━━━━━━━━━━━ 730s 20ms/step - loss: 9583.1992 - val_loss: 9318.7949


In [18]:
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

In [9]:
from tensorflow.keras.models import load_model

model.save('lstm_model.keras')
